## Import libraries

In [1]:
import re
import string
import spacy
import json
import pandas as pd
import numpy as np

## Load input

In [2]:
%cd D:/python/Toxic-comment-classification/kaggle/input/jigsaw-toxic-comment-classification-challenge

D:\python\Toxic-comment-classification\kaggle\input\jigsaw-toxic-comment-classification-challenge


In [3]:
data_train = pd.read_csv('train.csv.zip')
data_test = pd.read_csv('test.csv.zip')
test_label = pd.read_csv('test_labels.csv.zip')
sample_submission = pd.read_csv('sample_submission.csv.zip')

In [4]:
data_train['comment_text'].iloc[3097]

"You are a piece of Shiit \n\nYou are a pissant little creatin, who has no purpose in life. You graduated from a thirdclass college, but that's better than your prosti mom who's uneducated. You can't earn a thousandth of what Sweet Briar Alums earn. I saw your face, you are sooo ugly! You could never get into or pass at Sweet Briar with your monkey's brain!"

In [5]:
data_test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [6]:
sample_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [7]:
%cd D:/python/Toxic-comment-classification/text-preprocessing-tools-light

D:\python\Toxic-comment-classification\text-preprocessing-tools-light


In [8]:
with open("lemmatization-en.json") as file:
    lemmatizer = json.load(file)

with open("stopwords.txt") as file:
    stopwords = file.read().splitlines()

nlp = spacy.load("en_core_web_sm") 

## Cleaning

In [9]:
APPO = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "i would",
"i'd" : "i had",
"i'll" : "i will",
"i'm" : "i am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "i have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not",
"tryin'":"trying"
}

In [10]:
# nlp = spacy.load("en_core_web_sm")


def remove_special_characters(text):
    # Convert to lowercase
    text = text.lower()

    # Remove non ascii characters
    text = re.sub(r'[^\x00-\x7f]', r' ', text)

    # Remove emojis
    text = re.sub(r'[\U00010000-\U0010ffff]', ' ', text, flags=re.UNICODE)

    # Remove punctuation
    text = re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', ' ', text)

    # Remove \n\r
    text= re.sub(r'/\\n+|\\r+|\n+|\r+/', r' ', text)

    # Remove leaky elements like ip,user
    text=re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}","",text)

    # Removing usernames
    text=re.sub("\[\[.*\]","",text)

    # Remove all special characters
    # text = re.sub(r"[^\w']", ' ', text)
    return text

def split_numbers_from_characters(text):
    # Split numbers and characters
    parts = re.split('(\d+)', text)
    
    # If there was a number and a character part, add a space between them
    if len(parts) > 1:
        return ' '.join(parts)
    
    # If there was no number, return the original text
    return text

def remove_numbers(text):
    remove_digits = str.maketrans('', '', string.digits)
    return text.translate(remove_digits)

def clean_text_vanila(text):
    # Remove special characters
    text = remove_special_characters(text)
    text = remove_numbers(text)
    # Replace appos
    words=[APPO[word] if word in APPO else word for word in text.split()]
    return ' '.join(words)

def clean_text_light(text):
    # Remove special characters
    text = remove_special_characters(text)
    text = remove_numbers(text)
    # Replace appos
    words=[APPO[word] if word in APPO else word for word in text.split()]
    # lemmatization
    words = [lemmatizer.get(word, word) for word in words]
    # Remove numbers again as lemma might have numbers
    words = remove_numbers(' '.join(words)).split()
    # Remove stopwords
    words = [word for word in words if word not in stopwords]
    return ' '.join(words)

def clean_text_spacy(text):
    # Remove special characters
    text = remove_special_characters(text)
    text = split_numbers_from_characters(text)
    # Replace appos
    words=[APPO[word] if word in APPO else word for word in text.split()]
    # lemmatization
    words = [word.lemma_ for word in nlp(' '.join(words))]
    # Remove numbers again as lemma might have numbers
    words = remove_numbers(' '.join(words)).split()
    # Remove stopwords
    words = [word for word in words if word not in stopwords]
    return ' '.join(words)

In [11]:
clean_text_light('連絡 見学 __')

''

In [12]:
def write_to_file(data, filename):
    with open(filename, 'w') as f:
        for line in data:
            f.write(line + '\n')

def read_from_file(filename):
    with open(filename, 'r') as f:
        return f.read().splitlines()

In [13]:
%cd D:/python/Toxic-comment-classification/clean_data

D:\python\Toxic-comment-classification\clean_data


In [14]:
data_train_cleaned_vanilla = data_train['comment_text'].apply(clean_text_vanila)
data_test_cleaned_vanilla = data_test['comment_text'].apply(clean_text_vanila)
# labels = data_train.drop(['id', 'comment_text'], axis = 1)
write_to_file(data_train_cleaned_vanilla, 'data_train_cleaned_vanilla.txt')
write_to_file(data_test_cleaned_vanilla, 'data_test_cleaned_vanilla.txt')
# np.savetxt('data_train_cleaned_vanilla.txt', data_train_cleaned_vanilla.values, fmt='%s', newline='\n')
# labels.to_csv('labels.csv', index=False)

In [15]:
data_train_cleaned_light = data_train['comment_text'].apply(clean_text_light)
data_test_cleaned_light = data_test['comment_text'].apply(clean_text_light)
write_to_file(data_train_cleaned_light, 'data_train_cleaned_light.txt')
write_to_file(data_test_cleaned_light, 'data_test_cleaned_light.txt')

This next cell takes 44m to run

In [16]:
data_train_cleaned_spacy = data_train['comment_text'].apply(clean_text_spacy)
data_test_cleaned_spacy = data_test['comment_text'].apply(clean_text_spacy)
write_to_file(data_train_cleaned_spacy, 'data_train_cleaned_spacy.txt')
write_to_file(data_test_cleaned_spacy, 'data_test_cleaned_spacy.txt')